In [1]:
import matplotlib.pyplot as plt
import time

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
import numpy as np
from itertools import combinations

In [3]:
from pkg.tf_transform import *
from pkg.tf_robot import *
from pkg.constraint import *
from pkg.info import *
from pkg.tf_utils import *
from pkg.rotation_utils import *
from pkg.utils import *
from pkg.ur10 import *
from pkg.geometry import *
from pkg.collision import *
from pkg.distance import *

# Load urdf 

In [4]:
from urdf_parser_py.urdf import URDF
from pkg.ur10 import URDF_PATH, JOINT_NAMES, LINK_NAMES, ZERO_JOINT_POSE
from pkg.joint_utils import *
urdf_content = URDF.from_xml_file(URDF_PATH)

In [5]:
link_names = LINK_NAMES
base_name = LINK_NAMES[0]
joint_names = JOINT_NAMES
link_info_list = get_link_info_list(link_names, urdf_content)

# Visualize

In [6]:
from pkg.ros_rviz import *
pub, joints, rate = get_publisher(JOINT_NAMES)

gitem_list, gframe_dict= get_link_items_offsets(color=(0,1,0,0.5), display=True)
gitem_list += [
    GeometryItem(name='box1', gtype=GeoType.BOX, dims=[0.1,0.1,0.1], color=(0,1,0,1), display=True, collision=True)
]
gframe_dict.update({"box1":GeometryFrame(SE3(Rot_zyx(0,0,0),(0.5,0,0)), "world")
                   })
marker_list = set_markers(gitem_list, gframe_dict, urdf_content)
show_motion([np.array([0]*6)], marker_list, 
            [[gframe_dict[gitem.name] for gitem in gitem_list]], 
            pub, joints, error_skip=1e-6, period=1e-6)

Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker
publication OK - base_capsule
publication OK - shoulder_capsule
publication OK - upper_arm_capsule
publication OK - forearm_capsule
publication OK - wrist_1_capsule
publication OK - wrist_2_capsule
publication OK - wrist_3_capsule
publication OK - tool_mesh
publication OK - box1


# Constraint

In [7]:
class ObjectLayer(layers.Layer):
    def __init__(self, gitem, N_sim, *args, **kwargs):
        self.gitem, self.N_sim = gitem, N_sim
        super(ObjectLayer, self).__init__(*args, **kwargs)
        
    def set_frame(self, Toff_list, link_idx_list, N_link):
        self.Toff_list = tf.constant(Toff_list) # (N_sim, 4,4)
        self.link_one_hot = tf.reshape(tf.one_hot(link_idx_list, N_link), (N_sim,N_link,1,1)) # (N_sim, N_link)
        
    # 변수를 만듭니다.
    def build(self, input_shape):
        pass
    
    @tf.function
    def get_vertice(self):
        return self.vertice

    # call 메서드가 그래프 모드에서 사용되면
    # training 변수는 텐서가 됩니다.
    @tf.function
    def call(self, input=None):
        T_all = input # (N_sim, N_link, 4,4)
        T_act = K.sum(T_all*self.link_one_hot, axis=1) # (N_sim, 4,4)
        T_bo = tf.matmul(T_act, self.Toff_list) # (N_sim, 4,4)
        return T_bo

# Distance

In [8]:
class DistanceCalculator:
    def __init__(self, graph):
        self.graph = graph
        self.N_sim = graph.N_sim
        self.prepare_collision_dat()
            
    def prepare_collision_dat(self):
        pair_all = []
        for col_case in self.graph.pair_cases:
            setattr(self, "vtx1_"+col_case, self.graph.col_vtx1_dict[col_case])
            setattr(self, "vtx2_"+col_case, self.graph.col_vtx2_dict[col_case])
            setattr(self, "dist1_"+col_case, self.graph.col_dist1_dict[col_case])
            setattr(self, "dist2_"+col_case, self.graph.col_dist2_dict[col_case])
            setattr(self, "zeros_"+col_case, tf.constant(get_zero_points(self.N_sim, self.graph.N_col_dict[col_case]), dtype=tf.float32))
            setattr(self, "N_col_"+col_case, self.graph.N_col_dict[col_case])
            setattr(self, "flag_default_"+col_case, 
                    tf.constant(get_flag_default(self.N_sim, self.graph.N_col_dict[col_case]), dtype=tf.bool))
            setattr(self, "dist_default_"+col_case, 
                    tf.constant(get_dist_default(self.N_sim, self.graph.N_col_dict[col_case]), dtype=tf.float32))
            x_batch, y_batch = get_xy_batch(self.N_sim, self.graph.N_col_dict[col_case])
            setattr(self, "x_batch_"+col_case, 
                    tf.constant(x_batch, dtype=tf.float32))
            setattr(self, "y_batch_"+col_case, 
                    tf.constant(y_batch, dtype=tf.float32))
            setattr(self, "test_"+col_case, self.graph.col_vtx1_dict[col_case].shape[1]>0)
            
            pair_list = self.graph.col_pair_dict[col_case]
            pair_all += pair_list
        pair_all = np.array(pair_all)
        self.pair_obj1 = pair_all[:,0]
        self.pair_obj2 = pair_all[:,1]
            
    def apply_col_mask(self):
        for col_case in self.graph.pair_cases:
            setattr(self, "mask_"+col_case, self.graph.col_mask_dict[col_case])
    
    @tf.function
    def pt_pt_dist(self, Tbo_all_res):
        Tbo_all_res = tf.tile(Tbo_all_res, [1, self.N_col_pt_pt, 1,1,1,1])
        vtx1_all = tf.gather(K.sum(K.sum(Tbo_all_res*self.vtx1_pt_pt, axis=-1), axis=-3), [0,1,2], axis=-1)
        vtx2_all = tf.gather(K.sum(K.sum(Tbo_all_res*self.vtx2_pt_pt, axis=-1), axis=-3), [0,1,2], axis=-1)
        dist, vec, flag = distance_pt_pt(vtx1_all, vtx2_all, self.dist1_pt_pt, self.dist2_pt_pt)
        return dist, flag, vec, self.mask_pt_pt

    @tf.function
    def pt_ln_dist(self, Tbo_all_res):
        Tbo_all_res = tf.tile(Tbo_all_res, [1, self.N_col_pt_ln, 1,1,1,1])
        vtx1_all = tf.gather(K.sum(K.sum(Tbo_all_res*self.vtx1_pt_ln, axis=-1), axis=-3), [0,1,2], axis=-1)
        vtx2_all = tf.gather(K.sum(K.sum(Tbo_all_res*self.vtx2_pt_ln, axis=-1), axis=-3), [0,1,2], axis=-1)
        dist, vec, flag = distance_pt_ln(vtx1_all, vtx2_all, self.dist1_pt_ln, self.dist2_pt_ln)
        return dist, flag, vec, self.mask_pt_ln

    @tf.function
    def pt_pl_dist(self, Tbo_all_res):
        Tbo_all_res = tf.tile(Tbo_all_res, [1, self.N_col_pt_pl, 1,1,1,1])
        vtx1_all = tf.gather(K.sum(K.sum(Tbo_all_res*self.vtx1_pt_pl, axis=-1), axis=-3), [0,1,2], axis=-1)
        vtx2_all = tf.gather(K.sum(K.sum(Tbo_all_res*self.vtx2_pt_pl, axis=-1), axis=-3), [0,1,2], axis=-1)
        dist, vec, flag = distance_pt_pl(vtx1_all, vtx2_all, self.dist1_pt_pl, self.dist2_pt_pl, self.N_sim, self.N_col_pt_pl)
        return dist, flag, vec, self.mask_pt_pl

    @tf.function
    def pt_bx_dist(self, Tbo_all_res):
        Tbo_all_res = tf.tile(Tbo_all_res, [1, self.N_col_pt_bx, 1,1,1,1])
        vtx1_all = tf.gather(K.sum(K.sum(Tbo_all_res*self.vtx1_pt_bx, axis=-1), axis=-3), [0,1,2], axis=-1)
        vtx2_all = tf.gather(K.sum(K.sum(Tbo_all_res*self.vtx2_pt_bx, axis=-1), axis=-3), [0,1,2], axis=-1)
        dist, vec, flag = distance_pt_bx(vtx1_all, vtx2_all, self.dist1_pt_bx, self.dist2_pt_bx, self.N_sim, self.N_col_pt_bx)
        return dist, flag, vec, self.mask_pt_bx

    @tf.function
    def ln_ln_dist(self, Tbo_all_res):
        Tbo_all_res = tf.tile(Tbo_all_res, [1, self.N_col_ln_ln, 1,1,1,1])
        vtx1_all = tf.gather(K.sum(K.sum(Tbo_all_res*self.vtx1_ln_ln, axis=-1), axis=-3), [0,1,2], axis=-1)
        vtx2_all = tf.gather(K.sum(K.sum(Tbo_all_res*self.vtx2_ln_ln, axis=-1), axis=-3), [0,1,2], axis=-1)
        dist, vec, flag = distance_ln_ln(vtx1_all, vtx2_all, self.dist1_ln_ln, self.dist2_ln_ln, 
                                    self.N_sim, self.N_col_ln_ln, self.zeros_ln_ln)
        return dist, flag, vec, self.mask_ln_ln

    @tf.function
    def ln_pl_dist(self, Tbo_all_res):
        Tbo_all_res = tf.tile(Tbo_all_res, [1, self.N_col_ln_pl, 1,1,1,1])
        vtx1_all = tf.gather(K.sum(K.sum(Tbo_all_res*self.vtx1_ln_pl, axis=-1), axis=-3), [0,1,2], axis=-1)
        vtx2_all = tf.gather(K.sum(K.sum(Tbo_all_res*self.vtx2_ln_pl, axis=-1), axis=-3), [0,1,2], axis=-1)
        dist, vec, flag = distance_ln_pl(vtx1_all, vtx2_all, self.dist1_ln_pl, self.dist2_ln_pl, 
                                    self.N_sim, self.N_col_ln_pl, self.zeros_ln_pl)
        return dist, flag, vec, self.mask_ln_pl

    @tf.function
    def ln_bx_dist(self, Tbo_all_res):
        Tbo_all_res = tf.tile(Tbo_all_res, [1, self.N_col_ln_bx, 1,1,1,1])
        vtx1_all = tf.gather(K.sum(K.sum(Tbo_all_res*self.vtx1_ln_bx, axis=-1), axis=-3), [0,1,2], axis=-1)
        vtx2_all = tf.gather(K.sum(K.sum(Tbo_all_res*self.vtx2_ln_bx, axis=-1), axis=-3), [0,1,2], axis=-1)
        dist, vec, flag = distance_ln_bx(vtx1_all, vtx2_all, self.dist1_ln_bx, self.dist2_ln_bx, 
                                    self.N_sim, self.N_col_ln_bx, self.zeros_ln_bx
                                   )
        return dist, flag, vec, self.mask_ln_bx

    @tf.function
    def pl_pl_dist(self, Tbo_all_res):
        Tbo_all_res = tf.tile(Tbo_all_res, [1, self.N_col_pl_pl, 1,1,1,1])
        vtx1_all = tf.gather(K.sum(K.sum(Tbo_all_res*self.vtx1_pl_pl, axis=-1), axis=-3), [0,1,2], axis=-1)
        vtx2_all = tf.gather(K.sum(K.sum(Tbo_all_res*self.vtx2_pl_pl, axis=-1), axis=-3), [0,1,2], axis=-1)
        dist, vec, flag = distance_pl_pl(vtx1_all, vtx2_all, self.dist1_pl_pl, self.dist2_pl_pl, 
                                    self.flag_default_pl_pl, self.dist_default_pl_pl,
                                    self.x_batch_pl_pl, self.y_batch_pl_pl,
                                    4
                                   )
        return dist, flag, vec, self.mask_pl_pl

    @tf.function
    def pl_bx_dist(self, Tbo_all_res):
        Tbo_all_res = tf.tile(Tbo_all_res, [1, self.N_col_pl_bx, 1,1,1,1])
        vtx1_all = tf.gather(K.sum(K.sum(Tbo_all_res*self.vtx1_pl_bx, axis=-1), axis=-3), [0,1,2], axis=-1)
        vtx2_all = tf.gather(K.sum(K.sum(Tbo_all_res*self.vtx2_pl_bx, axis=-1), axis=-3), [0,1,2], axis=-1)
        dist, vec, flag = distance_pl_bx(vtx1_all, vtx2_all, self.dist1_pl_bx, self.dist2_pl_bx, 
                                    self.flag_default_pl_bx, self.dist_default_pl_bx,
                                    self.x_batch_pl_bx, self.y_batch_pl_bx,
                                    4
                                   )
        return dist, flag, vec, self.mask_pl_bx

    @tf.function
    def bx_bx_dist(self, Tbo_all_res):
        Tbo_all_res = tf.tile(Tbo_all_res, [1, self.N_col_bx_bx, 1,1,1,1])
        vtx1_all = tf.gather(K.sum(K.sum(Tbo_all_res*self.vtx1_bx_bx, axis=-1), axis=-3), [0,1,2], axis=-1)
        vtx2_all = tf.gather(K.sum(K.sum(Tbo_all_res*self.vtx2_bx_bx, axis=-1), axis=-3), [0,1,2], axis=-1)
        dist, vec, flag = distance_bx_bx(vtx1_all, vtx2_all, self.dist1_bx_bx, self.dist2_bx_bx, 
                                    self.flag_default_bx_bx, self.dist_default_bx_bx,
                                    self.x_batch_bx_bx, self.y_batch_bx_bx,
                                    4
                                   )
        return dist, flag, vec, self.mask_bx_bx
    
    @tf.function
    def calc_all(self, Tbo_all_res):
        dist_all = []
        flag_all = []
        vec_all = []
        mask_all = []
        if self.test_pt_pt:
            dist_pt_pt, flag_pt_pt, vec_pt_pt, mask_pt_pt = self.pt_pt_dist(Tbo_all_res)
            dist_all += [dist_pt_pt]
            flag_all += [flag_pt_pt]
            vec_all += [vec_pt_pt]
            mask_all += [mask_pt_pt]
            
        if self.test_pt_ln:
            dist_pt_ln, flag_pt_ln, vec_pt_ln, mask_pt_ln = self.pt_ln_dist(Tbo_all_res)
            dist_all += [dist_pt_ln]
            flag_all += [flag_pt_ln]
            vec_all += [vec_pt_ln]
            mask_all += [mask_pt_ln]
            
        if self.test_pt_pl:
            dist_pt_pl, flag_pt_pl, vec_pt_pl, mask_pt_pl = self.pt_pl_dist(Tbo_all_res)
            dist_all += [dist_pt_pl]
            flag_all += [flag_pt_pl]
            vec_all += [vec_pt_pl]
            mask_all += [mask_pt_pl]
            
        if self.test_pt_bx:
            dist_pt_bx, flag_pt_bx, vec_pt_bx, mask_pt_bx = self.pt_bx_dist(Tbo_all_res)
            dist_all += [dist_pt_bx]
            flag_all += [flag_pt_bx]
            vec_all += [vec_pt_bx]
            mask_all += [mask_pt_bx]
            
        if self.test_ln_ln:
            dist_ln_ln, flag_ln_ln, vec_ln_ln, mask_ln_ln = self.ln_ln_dist(Tbo_all_res)
            dist_all += [dist_ln_ln]
            flag_all += [flag_ln_ln]
            vec_all += [vec_ln_ln]
            mask_all += [mask_ln_ln]
            
        if self.test_ln_pl:
            dist_ln_pl, flag_ln_pl, vec_ln_pl, mask_ln_pl = self.ln_pl_dist(Tbo_all_res)
            dist_all += [dist_ln_pl]
            flag_all += [flag_ln_pl]
            vec_all += [vec_ln_pl]
            mask_all += [mask_ln_pl]
            
        if self.test_ln_bx:
            dist_ln_bx, flag_ln_bx, vec_ln_bx, mask_ln_bx = self.ln_bx_dist(Tbo_all_res)
            dist_all += [dist_ln_bx]
            flag_all += [flag_ln_bx]
            vec_all += [vec_ln_bx]
            mask_all += [mask_ln_bx]
            
        if self.test_pl_pl:
            dist_pl_pl, flag_pl_pl, vec_pl_pl, mask_pl_pl = self.pl_pl_dist(Tbo_all_res)
            dist_all += [dist_pl_pl]
            flag_all += [flag_pl_pl]
            vec_all += [vec_pl_pl]
            mask_all += [mask_pl_pl]
            
        if self.test_pl_bx:
            dist_pl_bx, flag_pl_bx, vec_pl_bx, mask_pl_bx = self.pl_bx_dist(Tbo_all_res)
            dist_all += [dist_pl_bx]
            flag_all += [flag_pl_bx]
            vec_all += [vec_pl_bx]
            mask_all += [mask_pl_bx]
            
        if self.test_bx_bx:
            dist_bx_bx, flag_bx_bx, vec_bx_bx, mask_bx_bx = self.bx_bx_dist(Tbo_all_res)
            dist_all += [dist_bx_bx]
            flag_all += [flag_bx_bx]
            vec_all += [vec_bx_bx]
            mask_all += [mask_bx_bx]

        dist_all = tf.concat(dist_all, axis=-2)
        flag_all = tf.concat(flag_all, axis=-2)
        vec_all = tf.concat(vec_all, axis=-2)
        mask_all = tf.concat(mask_all, axis=-2)
        
        return dist_all, flag_all, vec_all, mask_all
    
    @tf.function
    def jacobian(self, jac_o, vec_all):
        jac_op = tf.gather(jac_o, [0,1,2], axis=-1)
        jac_dp1 = tf.gather(jac_op, self.pair_obj1, axis=-3)
        jac_dp2 = tf.gather(jac_op, self.pair_obj2, axis=-3)
        vec_all = tf.expand_dims(vec_all, axis=-2)
        jac_d = K.sum(vec_all*(jac_dp1 - jac_dp2), axis=-1)
        return jac_d

# Graph

In [9]:
from collections import defaultdict

In [10]:
class GraphModel(tf.keras.Model):
    def __init__(self, robot_info, gitem_list, urdf_content, N_sim, 
                 alpha_jc=5, alpha_fc=200, alpha_jl=1, alpha_cl=1,alpha_cs=0, 
                 LIM=np.pi, LIM_BOUND=1e-1, COL_BOUND=1e-2, learning_rate=5e-3, col_iteration = 20):
        super(GraphModel, self).__init__()
        self.alpha_jc = alpha_jc
        self.alpha_fc = alpha_fc
        self.alpha_jl = alpha_jl
        self.alpha_cl = alpha_cl
        self.alpha_cs = alpha_cs
        self.N_sim = N_sim
        self.LIM = LIM
        self.LIM_BOUND = LIM_BOUND
        self.COL_BOUND = COL_BOUND
        self.learning_rate = learning_rate
        self.col_iteration = col_iteration
        self.robot_info = robot_info
        self.robot = RobotLayer(
            robot_info, dim=N_sim)
        self.joint_constraint = JointConstraintLoss(self.robot)
        self.frame_constraint = FrameConstraintLoss()
        self.robot_base = [robot_info.base_frame]*N_sim
        self.set_custom_loss(lambda *args: tf.constant(0.0), 0)
        self.link_name_list = self.robot.link_name_list
        self.link_idx_dict = self.robot.link_idx_dict
        self.adjacency_list = self.robot.adjacency_list
        self.adjacency_mat = self.robot.adjacency_mat
        self.dependency_list = self.robot.dependency_list
        self.dependency_mat = self.robot.dependency_mat
            
        self.object_dict = {}
        self.object_name_list = []
        self.object_vertice_list = []
        self.object_link_idx_list = []
        self.object_collision_flags = []
        for gitem in gitem_list:
            self.object_dict[gitem.name] = ObjectLayer(gitem, N_sim)
            self.object_collision_flags += [gitem.collision]
            self.object_name_list += [gitem.name]
            self.object_vertice_list += [np.pad(gitem.get_vertice(),((0,0),(0,1)),'constant', constant_values=(1))]
            self.object_link_idx_list += [0]
        self.num_object = len(self.object_name_list)
#         self.object_vertice_mat = tf.constant(self.object_vertice_list, dtype="float32")
        self.build_collision_combinations()
        self.set_collision_vtx()
        self.optimizer = tf.optimizers.SGD(learning_rate=learning_rate)

    def build_collision_combinations(self):
        obj_idx_list = np.arange(len(self.object_name_list))
        self.col_combinations = list(combinations(obj_idx_list,2))

        for i_c in range(len(self.col_combinations)):
            i_c_back = len(self.col_combinations)-1-i_c
            cmb = self.col_combinations[i_c_back]
            if not(self.object_collision_flags[cmb[0]] 
                   and self.object_collision_flags[cmb[1]]):
                del self.col_combinations[i_c_back]

        self.pair_cases = ['pt_pt', 'pt_ln', 'pt_pl', 'pt_bx',
                           'ln_ln', 'ln_pl', 'ln_bx',
                           'pl_pl', 'pl_bx','bx_bx']
        self.col_pair_dict = defaultdict(lambda:[])
        for comb in self.col_combinations:
            obj1 = self.object_dict[self.object_name_list[comb[0]]]
            obj2 = self.object_dict[self.object_name_list[comb[1]]]
            if obj1.gitem.gtype == GeoType.SPHERE:
                if obj2.gitem.gtype == GeoType.SPHERE:
                    self.col_pair_dict['pt_pt'] += [comb]
                elif obj2.gitem.gtype == GeoType.CYLINDER or obj2.gitem.gtype == GeoType.LINE:
                    self.col_pair_dict['pt_ln'] += [comb]
                elif obj2.gitem.gtype == GeoType.PLANE:
                    self.col_pair_dict['pt_pl'] += [comb]
                elif obj2.gitem.gtype == GeoType.BOX:
                    self.col_pair_dict['pt_bx'] += [comb]
            elif obj1.gitem.gtype == GeoType.CYLINDER or obj2.gitem.gtype == GeoType.LINE:
                if obj2.gitem.gtype == GeoType.SPHERE:
                    self.col_pair_dict['pt_ln'] += [(comb[1],comb[0])]
                elif obj2.gitem.gtype == GeoType.CYLINDER or obj2.gitem.gtype == GeoType.LINE:
                    self.col_pair_dict['ln_ln'] += [comb]
                elif obj2.gitem.gtype == GeoType.PLANE:
                    self.col_pair_dict['ln_pl'] += [comb]
                elif obj2.gitem.gtype == GeoType.BOX:
                    self.col_pair_dict['ln_bx'] += [comb]
            elif obj1.gitem.gtype == GeoType.PLANE:
                if obj2.gitem.gtype == GeoType.SPHERE:
                    self.col_pair_dict['pt_pl'] += [(comb[1],comb[0])]
                elif obj2.gitem.gtype == GeoType.CYLINDER or obj2.gitem.gtype == GeoType.LINE:
                    self.col_pair_dict['ln_pl'] += [(comb[1],comb[0])]
                elif obj2.gitem.gtype == GeoType.PLANE:
                    self.col_pair_dict['pl_pl'] += [comb]
                elif obj2.gitem.gtype == GeoType.BOX:
                    self.col_pair_dict['pl_bx'] += [comb]
            elif obj1.gitem.gtype == GeoType.BOX:
                if obj2.gitem.gtype == GeoType.SPHERE:
                    self.col_pair_dict['pt_bx'] += [(comb[1],comb[0])]
                elif obj2.gitem.gtype == GeoType.CYLINDER or obj2.gitem.gtype == GeoType.LINE:
                    self.col_pair_dict['ln_bx'] += [(comb[1],comb[0])]
                elif obj2.gitem.gtype == GeoType.PLANE:
                    self.col_pair_dict['pl_bx'] += [(comb[1],comb[0])]
                elif obj2.gitem.gtype == GeoType.BOX:
                    self.col_pair_dict['bx_bx'] += [comb]
            
    def assign_frame_dict(self, gframeset_list):
        frame_dict = {k: [] for k in self.object_dict.keys()}
        link_dict = {k: [] for k in self.object_dict.keys()}
        for gframeset in gframeset_list:
            for k, gframe in gframeset.items():
                frame_dict[k] += [gframe.Toff]
                link_dict[k] += [self.robot.link_name_list.index(gframe.link_name)]
        for k in frame_dict.keys():
            self.object_dict[k].set_frame(np.array(frame_dict[k]), np.array(link_dict[k]), self.robot.num_link)
            i_obj = self.get_object_index(k)
            self.object_link_idx_list[i_obj] = np.array(link_dict[k])
        self.object_link_idx_mat = np.transpose(self.object_link_idx_list)
        self.apply_col_mask()

        self.object_depend_mask = tf.reshape(
            tf.gather(self.robot.mask_depend, self.object_link_idx_mat, axis=-3), 
            (self.N_sim, self.num_object, self.robot.DOF, 1))
            
    def get_object_index(self, name):
        return graph.object_name_list.index(name)
            
    @tf.function
    def assign_Q(self, Q):
        self.robot.assign_Q(Q)
            
    @tf.function
    def get_Q(self):
        return self.robot.get_Q()

    @tf.function
    def call(self, inputs=None):
        T_all = self.robot(self.robot_base)
        Tbo_all = []
        for obj_name in self.object_name_list:
            Tbo_all += [self.object_dict[obj_name](T_all)] #(Nobj,N_sim,4,4)
        Tbo_all = K.stack(Tbo_all, axis=1) #(N_sim,Nobj,4,4)
        return T_all, Tbo_all
    
    @tf.function
    def calc_joint_limit(self):
        Q = self.get_Q()
        return K.sum(1/((K.min(((self.LIM-Q), (self.LIM+Q)), axis=-1)/self.LIM_BOUND)**3))
    
    @tf.function
    def calc_loss(self, T_all, Tbo_all, Qtar, binQ, Ttar, binT):
        jl_loss = self.calc_joint_limit()
        jc_loss = self.joint_constraint((Qtar, binQ))
        fc_loss = self.frame_constraint((T_all[:,-1,:,:],Ttar, binT))
        cl_loss = self.calc_collision_loss(T_all, Tbo_all)
        return self.alpha_jc*jc_loss + self.alpha_fc*fc_loss + \
                self.alpha_jl*jl_loss + self.alpha_cs*self.calc_custom_loss(T_all, Tbo_all) \
                + self.alpha_cl*cl_loss 
                
    
    @tf.function
    def forward(self, Qtar, binQ, Ttar, binT):
        T_all, Tbo_all= graph(None)
        loss = self.calc_loss(T_all, Tbo_all, Qtar, binQ, Ttar, binT)
        return loss
    
    @tf.function
    def update_once(self, Qtar, binQ, Ttar, binT, max_gradient):
        with tf.GradientTape() as g:
            # Forward pass.
            loss = self.forward(Qtar, binQ, Ttar, binT)

        # Variables to update, i.e. trainable variables.
        trainable_variables = self.trainable_variables

        # Compute gradients.
        gradients = g.gradient(loss, trainable_variables)
        gradients = tf.unstack(clip_gradient(gradients, max_gradient))
    
        # Update W and b following gradients.
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        return loss
    
    def set_custom_loss(self, custom_loss, alpha_cs, *args, **kwargs):
        self.custom_loss = custom_loss
        self.alpha_cs = alpha_cs
        self.cl_args = args
        self.cl_kwargs = kwargs
        
    def calc_custom_loss(self, T_all, Tbo_all):
        return self.custom_loss(self, T_all, Tbo_all, *self.cl_args, **self.cl_kwargs)

    @tf.function
    def test_collision(self, T_all, Tbo_all):
        Pbo_all = tf.gather(tf.gather(Tbo_all, [0,1,2], axis=-2), 3, axis=-1)
        col_combs_masked_batch = self.get_collision_combination()
        batch_flat = tf.repeat(np.reshape(np.arange(self.N_sim), (self.N_sim, 1)), (self.N_col,)*self.N_sim,axis=0)
        col_combs_masked_flat = tf.concat([batch_flat, col_combs_masked_batch], axis=-1)
        vtx_tf = tf.gather(
            K.sum(tf.expand_dims(Tbo_all, axis=-3)*tf.expand_dims(self.object_vertice_mat, axis=-2), axis=-1), 
            [0,1,2], axis=-1)
        vtx_tf_1 = tf.reshape(tf.gather_nd(vtx_tf, tf.gather(col_combs_masked_flat, [0,1], axis=1)), (self.N_sim, self.N_col, -1,3))
        vtx_tf_2 = tf.reshape(tf.gather_nd(vtx_tf, tf.gather(col_combs_masked_flat, [0,2], axis=1)), (self.N_sim, self.N_col, -1,3))
        Pbo_all_1 = tf.reshape(tf.gather_nd(Pbo_all, tf.gather(col_combs_masked_flat, [0,1], axis=1)), (self.N_sim, self.N_col,3))
        Pbo_all_2 = tf.reshape(tf.gather_nd(Pbo_all, tf.gather(col_combs_masked_flat, [0,2], axis=1)), (self.N_sim, self.N_col,3))
        v_batch = tf.expand_dims(Pbo_all_1-Pbo_all_2, axis=-2)
        v_batch = tf.stop_gradient(tf.linalg.cross(v_batch, self.x_batch)+tf.linalg.cross(v_batch, self.y_batch))
        dist, flag = test_collision_batch(vtx_tf_1, vtx_tf_2, 
                             v_batch, self.flag_default, self.dist_default, IterationAllowed=self.col_iteration)
        return dist, flag, col_combs_masked_flat 
    
    @tf.function
    def calc_collision_loss(self, T_all, Tbo_all):
        dist, flag, _  = self.test_collision(T_all, Tbo_all)
        return K.sum(1/((dist/self.COL_BOUND)**3))
    
    def get_collision_combination_names(self):
        comb_names = []
        for comb in self.get_collision_combination():
            comb_names += ["{} - {}".format(self.object_name_list[comb[0]], graph.object_name_list[comb[1]])]
            
    def get_vtx_pair(self, pair_list, N_vtx1, N_vtx2):
        N_col = len(pair_list)
        pair_vtx1 = np.zeros((1, len(pair_list), len(self.object_name_list), N_vtx1, 1, 4)) # (N_sim, N_col, N_obj, N_vtx, 1, 4)
        pair_vtx2 = np.zeros((1, len(pair_list), len(self.object_name_list), N_vtx2, 1, 4))
        pair_dist1 = np.zeros((1, len(pair_list), 1))
        pair_dist2 = np.zeros((1, len(pair_list), 1))
        for i_c, comb in zip(range(len(pair_list)), pair_list):
            pair_vtx1[0,i_c,comb[0],:,0] = np.pad(self.object_dict[self.object_name_list[comb[0]]].gitem.get_vertice(),((0,0),(0,1)),'constant', constant_values=(1))
            pair_vtx2[0,i_c,comb[1],:,0] = np.pad(self.object_dict[self.object_name_list[comb[1]]].gitem.get_vertice(),((0,0),(0,1)),'constant', constant_values=(1))
            pair_dist1[0,i_c, 0] = self.object_dict[self.object_name_list[comb[0]]].gitem.get_radius()
            pair_dist2[0,i_c, 0] = self.object_dict[self.object_name_list[comb[1]]].gitem.get_radius()
        pair_vtx1 = tf.constant(pair_vtx1, dtype=tf.float32)
        pair_vtx2 = tf.constant(pair_vtx2, dtype=tf.float32)
        pair_dist1 = tf.constant(pair_dist1, dtype=tf.float32)
        pair_dist2 = tf.constant(pair_dist2, dtype=tf.float32)
        return pair_vtx1, pair_vtx2, pair_dist1, pair_dist2, N_col
    
    def set_collision_vtx(self):
        self.col_vtx1_dict = {}
        self.col_vtx2_dict = {}
        self.col_dist1_dict = {}
        self.col_dist2_dict = {}
        self.N_col_dict = {}
        self.N_vtx_dict = {"pt":1, "ln":2, "pl":4, "bx":8}
        for col_case in self.pair_cases:
            self.col_vtx1_dict[col_case], self.col_vtx2_dict[col_case], \
                self.col_dist1_dict[col_case], self.col_dist2_dict[col_case], self.N_col_dict[col_case] = \
                    self.get_vtx_pair(
                        self.col_pair_dict[col_case], self.N_vtx_dict[col_case[:2]], self.N_vtx_dict[col_case[3:]])
        self.dcal = DistanceCalculator(self)
        
    def apply_col_mask(self):
        self.col_mask_dict = {}
        for col_case in self.pair_cases:
            N_col = self.N_col_dict[col_case]
            mask = np.zeros((self.N_sim, N_col, 1), dtype=np.float32)
            for i_col, comb in zip(range(N_col), self.col_pair_dict[col_case]):
                mask[:, i_col, 0] = tf.gather_nd(self.adjacency_mat, self.object_link_idx_mat[:, comb])
            self.col_mask_dict[col_case] = 1-mask
        self.dcal.apply_col_mask()
        
    def jacobian_object(self, Tbo_all):
        RP_all = tf.gather(Tbo_all, [0,1,2], axis=-2) # N_sim, N_obj, 3,4
        R_all = tf.gather(RP_all, [0,1,2], axis=-1)
        P_all = tf.gather(RP_all, [3], axis=-1)
        R_jnt = tf.gather(R_all, self.robot.idx_var, axis=-3)
        P_jnt = tf.gather(P_all, self.robot.idx_var, axis=-3)
        axis_jnt = K.sum(R_jnt*self.robot._axis_prism, axis=-1)
        axis_jnt_rep = tf.tile(tf.expand_dims(axis_jnt, axis=-3), [1,self.num_object,1,1])

        jac_prism_ = tf.pad(axis_jnt, [[0,0], [0,0], [0,3]]) # N_sim, DOF, 6
        jac_prism = tf.expand_dims(jac_prism_, axis=-3) * self.object_depend_mask# N_sim, N_obj, DOF, 6

        pij = tf.reshape(tf.expand_dims(P_all, axis=-3)-tf.expand_dims(P_jnt, axis=-4), 
                         (self.N_sim, self.num_object, self.robot.DOF, 3)) # N_sim, N_link, DOF, 3, 1
        jac_p_rev = tf.linalg.cross(pij, axis_jnt_rep)
        jac_revol = tf.concat([jac_p_rev, axis_jnt_rep], axis=-1) * self.object_depend_mask# N_sim, N_link, DOF, 6

        jacobian = self.robot.mask_prism*jac_prism + self.robot.mask_revol*jac_revol
        return jacobian
    
    def jacobian(self, T_all, Tbo_all):
        return self.robot.jacobian(T_all), self.jacobian_object(Tbo_all)
        

In [11]:
N_sim = 50
N_joints = 9
DOF = 6
gtimer = GlobalTimer()


gtimer.tic("initialize")
robot_info = RobotInfo(link_info_list, rname = "rbt1", base_frame=np.identity(4,dtype=np.float32))
graph = GraphModel(robot_info=robot_info, gitem_list=gitem_list, urdf_content=urdf_content, 
                   N_sim=N_sim, learning_rate=5e-3, 
                   alpha_cl=0.001)
Q_ = np.array([tuple(ZERO_JOINT_POSE+(np.random.rand(DOF)*2-1)*np.pi/100) for _ in range(N_sim)], dtype=np.float32)
Ttar_ = SE3(Rot_zyx(0,0,np.pi),(0.5,0,0.00)).astype(np.float32)
gframe_dict_list = [gframe_dict]*N_sim
graph.assign_Q(Q_)
graph.assign_frame_dict(gframe_dict_list)
T_all, Tbo_all = graph(None)
Tbo_all_res = tf.reshape(Tbo_all, (graph.N_sim, 1, graph.num_object, 1, 4,4))
res = graph.dcal.calc_all(Tbo_all_res)
gtimer.toc("initialize")
print(gtimer)

initialize: 	5120.0 ms/1 = 5119.558 ms 



In [12]:
gtimer.reset()
Q_ = np.array([tuple(ZERO_JOINT_POSE+(np.random.rand(DOF)*2-1)*np.pi/100) for _ in range(N_sim)], dtype=np.float32)
Ttar_ = SE3(Rot_zyx(0,0,np.pi),(0.5,0,0.00)).astype(np.float32)
gtimer.tic("assign")
graph.assign_Q(Q_)
graph.assign_frame_dict(gframe_dict_list)
gtimer.toc("assign")
print(gtimer)

assign: 	6.0 ms/1 = 5.77 ms 



In [15]:
Q_ = np.array([(0, 0,)+tuple(ZERO_JOINT_POSE+(np.random.rand(DOF)*2-1)*np.pi/100)+(0,) for _ in range(N_sim)], dtype=np.float32)
Ttar_ = SE3(Rot_zyx(0,0,np.pi),(0.5,0,0.00)).astype(np.float32)

gtimer.reset()
res_vec = []
for _ in range(100):
    gtimer.tic("calc")
    T_all, Tbo_all = graph(None)
    gtimer.toc("calc")
    gtimer.tic("jacobian")
    jac_r, jac_o = graph.jacobian(T_all, Tbo_all)
    gtimer.toc("jacobian")
    gtimer.tic("collision")
    Tbo_all_res = tf.reshape(Tbo_all, (graph.N_sim, 1, graph.num_object, 1, 4,4))
    dist_all, flag_all, vec_all, mask_all = graph.dcal.calc_all(Tbo_all_res)
    gtimer.toc("collision")
    gtimer.tic("col_jacobian")
    jac_d = graph.dcal.jacobian(jac_o, vec_all)
    gtimer.toc("col_jacobian")
    res_vec += [res]
print(gtimer)

calc: 	215.0 ms/100 = 2.151 ms 
jacobian: 	157.0 ms/100 = 1.573 ms 
collision: 	526.0 ms/100 = 5.257 ms 
col_jacobian: 	41.0 ms/100 = 0.408 ms 



# Test full loop

In [16]:
@tf.function
def _loop(value):
    T_all, Tbo_all = graph(None)
    Tbo_all_res = tf.reshape(Tbo_all, (graph.N_sim, 1, graph.num_object, 1, 4,4))
    jac_r, jac_o = graph.jacobian(T_all, Tbo_all)
    dist_all, flag_all, vec_all, mask_all = graph.dcal.calc_all(Tbo_all_res)
    jac_d = graph.dcal.jacobian(jac_o, vec_all)
    return K.sum(dist_all)*K.sum(mask_all)+K.sum(jac_r)+K.sum(jac_o)+K.sum(jac_d)
@tf.function
def run_test(N_loop):
    res = tf.while_loop(
        lambda value: True, _loop, (tf.constant(0.0),), 
        parallel_iterations=10, maximum_iterations=N_loop
    )
    return res
run_test(100)

(<tf.Tensor: shape=(), dtype=float32, numpy=434183.62>,)

In [18]:
gtimer.reset()
gtimer.tic("loop100")
res = run_test(100)
gtimer.toc("loop100")
print(gtimer)

loop100: 	474.0 ms/1 = 474.198 ms 



# Gradient Clipping

In [31]:
gtimer.reset()
gtimer.tic("func")
with tf.GradientTape() as g:
    # Forward pass.
    T_all, Tbo_all = graph(None)
    Tbo_all_res = tf.reshape(Tbo_all, (graph.N_sim, 1, graph.num_object, 1, 4,4))
    dist, flag, mask = graph.dcal.calc_all(Tbo_all_res)
    loss = dist
#     loss = K.sum(dist[6])
# Variables to update, i.e. trainable variables.
trainable_variables = graph.trainable_variables

gtimer.toc("func")
gtimer.tic("grad")
# Compute gradients.
# gradients = g.gradient(loss, trainable_variables)
# jacobian = g.batch_jacobian(loss, trainable_variables[0])
gtimer.toc("grad")
# gradients = tf.unstack(clip_gradient(gradients, max_gradient))

# # Update W and b following gradients.
gtimer.tic("apply")
# graph.optimizer.apply_gradients(zip(gradients, graph.trainable_variables))
gtimer.toc("apply")
print(gtimer)

func: 	3054.0 ms/1 = 3053.651 ms 
grad: 	0.0 ms/1 = 0.02 ms 
apply: 	0.0 ms/1 = 0.016 ms 



In [32]:
gtimer.reset()
gtimer.tic("batch_jacobian")
# bjac = g.batch_jacobian(dist, graph.trainable_variables[0])
gtimer.toc("batch_jacobian")
print(gtimer)

batch_jacobian: 	0.0 ms/1 = 0.019 ms 



# test full gradient

In [20]:
@tf.function
def test_grad(res):
    with tf.GradientTape() as g:
        # Forward pass.
        T_all, Tbo_all = graph(None)
        Tbo_all_res = tf.reshape(Tbo_all, (graph.N_sim, 1, graph.num_object, 1, 4,4))
        dist, flag, mask = graph.dcal.calc_all(Tbo_all_res)
#         dist = dist*mask
#         loss = [tf.gather(dist, i_sim) for i_sim in range(N_sim)]
    #     loss = K.sum(dist[6])
    # Variables to update, i.e. trainable variables

    # Compute gradients.
    jacobian = g.batch_jacobian(dist, graph.trainable_variables[0])
    return K.sum(jacobian)
#     bjac = g.batch_jacobian(dist, graph.trainable_variables[0])
#     return bjac
gradients = test_grad(0)

ERROR:tensorflow:Got error while pfor was converting op name: "gradients/PartitionedCall_grad/PartitionedCall_146"
op: "PartitionedCall"
input: "gradients/grad_ys_0"
input: "gradients/zeros_like"
input: "gradients/PartitionedCall_grad/PartitionedCall"
input: "gradients/PartitionedCall_grad/PartitionedCall_1"
input: "gradients/PartitionedCall_grad/PartitionedCall_2"
input: "gradients/PartitionedCall_grad/PartitionedCall_3"
input: "gradients/PartitionedCall_grad/PartitionedCall_4"
input: "gradients/PartitionedCall_grad/PartitionedCall_5"
input: "gradients/PartitionedCall_grad/PartitionedCall_6"
input: "gradients/PartitionedCall_grad/PartitionedCall_7"
input: "gradients/PartitionedCall_grad/PartitionedCall_8"
input: "gradients/PartitionedCall_grad/PartitionedCall_9"
input: "gradients/PartitionedCall_grad/PartitionedCall_10"
input: "gradients/PartitionedCall_grad/PartitionedCall_11"
input: "gradients/PartitionedCall_grad/PartitionedCall_12"
input: "gradients/PartitionedCall_grad/Partitione

ERROR:tensorflow:name: "gradients/PartitionedCall_grad/PartitionedCall_146"
op: "PartitionedCall"
input: "gradients/grad_ys_0"
input: "gradients/zeros_like"
input: "gradients/PartitionedCall_grad/PartitionedCall"
input: "gradients/PartitionedCall_grad/PartitionedCall_1"
input: "gradients/PartitionedCall_grad/PartitionedCall_2"
input: "gradients/PartitionedCall_grad/PartitionedCall_3"
input: "gradients/PartitionedCall_grad/PartitionedCall_4"
input: "gradients/PartitionedCall_grad/PartitionedCall_5"
input: "gradients/PartitionedCall_grad/PartitionedCall_6"
input: "gradients/PartitionedCall_grad/PartitionedCall_7"
input: "gradients/PartitionedCall_grad/PartitionedCall_8"
input: "gradients/PartitionedCall_grad/PartitionedCall_9"
input: "gradients/PartitionedCall_grad/PartitionedCall_10"
input: "gradients/PartitionedCall_grad/PartitionedCall_11"
input: "gradients/PartitionedCall_grad/PartitionedCall_12"
input: "gradients/PartitionedCall_grad/PartitionedCall_13"
input: "gradients/Partitioned

ERROR:tensorflow:Got error while pfor was converting op name: "gradients/PartitionedCall_6_grad/PartitionedCall"
op: "PartitionedCall"
input: "gradients/concat_3_grad/Slice_1"
input: "gradients/concat_5_grad/Slice_1"
input: "gradients/PartitionedCall_6_grad/PartitionedCall_6"
input: "gradients/PartitionedCall_6_grad/PartitionedCall_6_1"
input: "gradients/PartitionedCall_6_grad/PartitionedCall_6_2"
input: "gradients/PartitionedCall_6_grad/PartitionedCall_6_3"
input: "gradients/PartitionedCall_6_grad/PartitionedCall_6_4"
input: "gradients/PartitionedCall_6_grad/PartitionedCall_6_5"
input: "gradients/PartitionedCall_6_grad/PartitionedCall_6_6"
input: "gradients/PartitionedCall_6_grad/PartitionedCall_6_7"
input: "gradients/PartitionedCall_6_grad/PartitionedCall_6_8"
input: "gradients/PartitionedCall_6_grad/PartitionedCall_6_9"
input: "gradients/PartitionedCall_6_grad/PartitionedCall_6_10"
input: "gradients/PartitionedCall_6_grad/PartitionedCall_6_11"
input: "gradients/PartitionedCall_6_gra

ERROR:tensorflow:name: "gradients/PartitionedCall_6_grad/PartitionedCall"
op: "PartitionedCall"
input: "gradients/concat_3_grad/Slice_1"
input: "gradients/concat_5_grad/Slice_1"
input: "gradients/PartitionedCall_6_grad/PartitionedCall_6"
input: "gradients/PartitionedCall_6_grad/PartitionedCall_6_1"
input: "gradients/PartitionedCall_6_grad/PartitionedCall_6_2"
input: "gradients/PartitionedCall_6_grad/PartitionedCall_6_3"
input: "gradients/PartitionedCall_6_grad/PartitionedCall_6_4"
input: "gradients/PartitionedCall_6_grad/PartitionedCall_6_5"
input: "gradients/PartitionedCall_6_grad/PartitionedCall_6_6"
input: "gradients/PartitionedCall_6_grad/PartitionedCall_6_7"
input: "gradients/PartitionedCall_6_grad/PartitionedCall_6_8"
input: "gradients/PartitionedCall_6_grad/PartitionedCall_6_9"
input: "gradients/PartitionedCall_6_grad/PartitionedCall_6_10"
input: "gradients/PartitionedCall_6_grad/PartitionedCall_6_11"
input: "gradients/PartitionedCall_6_grad/PartitionedCall_6_12"
input: "gradien

ERROR:tensorflow:Got error while pfor was converting op name: "loop_body/PartitionedCall"
op: "PartitionedCall"
input: "gradient_tape/Reshape"
input: "loop_body/zeros_1"
input: "PartitionedCall:3"
input: "PartitionedCall:4"
input: "PartitionedCall:5"
input: "PartitionedCall:6"
input: "PartitionedCall:7"
input: "PartitionedCall:8"
input: "PartitionedCall:9"
input: "PartitionedCall:10"
input: "PartitionedCall:11"
input: "PartitionedCall:12"
input: "PartitionedCall:13"
input: "PartitionedCall:14"
input: "PartitionedCall:15"
input: "PartitionedCall:16"
input: "PartitionedCall:17"
input: "PartitionedCall:18"
input: "PartitionedCall:19"
input: "PartitionedCall:20"
input: "PartitionedCall:21"
input: "PartitionedCall:22"
input: "PartitionedCall:23"
input: "PartitionedCall:24"
input: "PartitionedCall:25"
input: "PartitionedCall:26"
input: "PartitionedCall:27"
input: "PartitionedCall:28"
input: "PartitionedCall:29"
input: "PartitionedCall:30"
input: "PartitionedCall:31"
input: "PartitionedCall:3

ERROR:tensorflow:name: "loop_body/PartitionedCall"
op: "PartitionedCall"
input: "gradient_tape/Reshape"
input: "loop_body/zeros_1"
input: "PartitionedCall:3"
input: "PartitionedCall:4"
input: "PartitionedCall:5"
input: "PartitionedCall:6"
input: "PartitionedCall:7"
input: "PartitionedCall:8"
input: "PartitionedCall:9"
input: "PartitionedCall:10"
input: "PartitionedCall:11"
input: "PartitionedCall:12"
input: "PartitionedCall:13"
input: "PartitionedCall:14"
input: "PartitionedCall:15"
input: "PartitionedCall:16"
input: "PartitionedCall:17"
input: "PartitionedCall:18"
input: "PartitionedCall:19"
input: "PartitionedCall:20"
input: "PartitionedCall:21"
input: "PartitionedCall:22"
input: "PartitionedCall:23"
input: "PartitionedCall:24"
input: "PartitionedCall:25"
input: "PartitionedCall:26"
input: "PartitionedCall:27"
input: "PartitionedCall:28"
input: "PartitionedCall:29"
input: "PartitionedCall:30"
input: "PartitionedCall:31"
input: "PartitionedCall:32"
input: "PartitionedCall:33"
input: "

StagingError: in user code:

    <ipython-input-20-2c4d78f41612>:14 test_grad  *
        jacobian = g.batch_jacobian(dist, graph.trainable_variables[0])
    /home/junsu/.local/lib/python3.6/site-packages/tensorflow/python/ops/parallel_for/pfor.py:3600 f  *
        [converter._convert_helper(x).t for x in func._func_graph_outputs])
    /home/junsu/.local/lib/python3.6/site-packages/tensorflow/python/ops/parallel_for/pfor.py:3600 f  *
        [converter._convert_helper(x).t for x in func._func_graph_outputs])
    /home/junsu/.local/lib/python3.6/site-packages/tensorflow/python/ops/parallel_for/pfor.py:3600 f  *
        [converter._convert_helper(x).t for x in func._func_graph_outputs])
    /home/junsu/.local/lib/python3.6/site-packages/tensorflow/python/ops/parallel_for/pfor.py:1457 _convert_helper  **
        if flags.FLAGS.op_conversion_fallback_to_while_loop:
    /home/junsu/.local/lib/python3.6/site-packages/tensorflow/python/platform/flags.py:85 __getattr__
        wrapped(_sys.argv

In [ ]:
@ tf.function
def loop_grad():
    N_loop=100
    res = tf.while_loop(
        lambda value: True, test_grad, (tf.constant(0.0),), 
        parallel_iterations=10, maximum_iterations=N_loop
    )
    return res
loop_grad()

In [ ]:
gtimer.reset()
gtimer.tic("func")
jacobian = loop_grad()
gtimer.toc("func")
# gradients = tf.unstack(clip_gradient(gradients, max_gradient))

# # Update W and b following gradients.
gtimer.tic("apply")
# graph.optimizer.apply_gradients(zip(gradients, graph.trainable_variables))
gtimer.toc("apply")
print(gtimer)
# print(gradients.shape)

# SGD Optimizer

In [ ]:
Q_ = np.array([(0, 0,)+tuple(ZERO_JOINT_POSE+(np.random.rand(DOF)*2-1)*np.pi/100)+(0,) for _ in range(N_sim)], dtype=np.float32)
Ttar_ = SE3(Rot_zyx(0,0,np.pi),(0.5,0,0.00)).astype(np.float32)
gframe_dict_list = [gframe_dict]*N_sim
graph.assign_Q(Q_)
graph.assign_frame_dict(gframe_dict_list)

In [ ]:
time_vec = []
Q_list = [list(map(lambda x: x.numpy(), graph.get_Q()))]
# Tbo_all_list = [np.array([[gframe_dict[gitem.name].Toff for gitem in gitem_list]]*N_sim)]
N_iter = 100
# Run training for the given number of steps.
Qtar, binQ, Ttar, binT = (np.zeros((N_sim, N_joints), dtype='float32'), np.zeros(N_sim, dtype='float32'),
                      np.array([Ttar_]*N_sim), np.ones(N_sim, dtype='float32'))
dist_list = []
loss_list = []
jl_loss_list = []
jc_loss_list = []
fc_loss_list = []
cl_loss_list = []
gradients_list = []
gtimer.reset()
for _ in range(N_iter):
    # Run the optimization to update W and b values.
    T_all, Tbo_all = graph(None)
    dist_list += [K.min(graph.test_collision(T_all, Tbo_all)[0]).numpy()]
    jl_loss = graph.calc_joint_limit()
    jc_loss = graph.joint_constraint((Qtar, binQ))
    fc_loss = graph.frame_constraint((T_all[:,-1,:,:],Ttar, binT))
    cl_loss = graph.calc_collision_loss(T_all, Tbo_all)
    gtimer.tic("update")
    loss = graph.update_once(Qtar, binQ, Ttar, binT, max_gradient=10)
#     max_gradient=10

#     with tf.GradientTape() as g:
#         # Forward pass.
#         loss = graph.forward(Qtar, binQ, Ttar, binT)

#     # Variables to update, i.e. trainable variables.
#     trainable_variables = graph.trainable_variables

#     # Compute gradients.
#     gradients = g.gradient(loss, trainable_variables)
#     gradients = tf.unstack(clip_gradient(gradients, max_gradient))

#     # Update W and b following gradients.
#     graph.optimizer.apply_gradients(zip(gradients, graph.trainable_variables))
    gtimer.toc("update")
    gtimer.tic("record")
    Q_list += [graph.get_Q().numpy()]
    loss_list += [loss]
#     gradients_list += [gradients]
    jl_loss_list += [jl_loss]
    jc_loss_list += [jc_loss]
    fc_loss_list += [fc_loss]
    cl_loss_list += [cl_loss]
    gtimer.toc("record")
    if isnan(dist_list[-1]) or dist_list[-1]<0:
        break
gtimer.print_time_log()
print("loss: {}".format(loss))

# Plot result

In [ ]:
Q_all = np.array(Q_list)
Q_all_i = np.array([Q_all[i_iter, i_sim,:] for i_iter in range(len(Q_list))])
plt.plot(Q_all_i,'-o')
plt.plot(Q_all_i[1:,:]-Q_all_i[:-1,:],'-.')

In [ ]:
plt.plot(dist_list, label="distance")

In [ ]:
plt.plot(loss_list, label="total")
plt.plot(np.multiply(jl_loss_list, graph.alpha_jl), label="joint limit")
plt.plot(np.multiply(jc_loss_list, graph.alpha_jc), label="joint target")
plt.plot(np.multiply(fc_loss_list, graph.alpha_fc), label="frame target")
plt.plot(np.multiply(cl_loss_list, graph.alpha_cl), label="collision dist")
plt.legend()
axes = plt.gca()
# axes.set_ylim([-1e1,5e1])
# axes.set_xlim([0,1e1])

# Show rviz

In [20]:
q = Q_[0]
gframevec = [gframe_dict[gitem.name] for gitem in gitem_list]
pose_list = [q[2:-1]]
gframevec_list = [gframevec]
show_motion(pose_list, marker_list, gframevec_list, pub, joints, error_skip=1e-6, period=1e-6)

# Display motion

In [ ]:
for i_sim in range(1): # N_sim):
    for i_iter in range(0,len(Q_list)):
        q = Q_list[i_iter][i_sim]
        gframevec = [gframe_dict[gitem.name] for gitem in gitem_list]
        pose_list = [q[2:-1]]
        gframevec_list = [gframevec]
        show_motion(pose_list, marker_list, gframevec_list, pub, joints, error_skip=1e-6, period=1e-6)
        time.sleep(10e-2)

# Collision error case 

In [ ]:
Q_error = np.loadtxt("Q_error.csv",delimiter=",",dtype=np.float32)
graph.assign_Q(Q_error)
graph.assign_frame_dict(gframe_dict_list)

T_all, Tbo_all = graph(None)
dist = K.min(graph.test_collision(T_all, Tbo_all)[0]).numpy()
cl_loss = graph.calc_collision_loss(T_all, Tbo_all)
print("dist: {}".format(dist))

In [ ]:
q = Q_error[0]
gframevec = [gframe_dict[gitem.name] for gitem in gitem_list]
pose_list = [q[2:-1]]
gframevec_list = [gframevec]
show_motion(pose_list, marker_list, gframevec_list, pub, joints, error_skip=1e-6, period=1e-6)

# Plot 3D

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d as mplot3d

In [ ]:
fig = plt.figure()
sub = fig.add_subplot(1,1,1,projection="3d")
sub.plot(x,y,z)
sub.set_xlabel('x')
sub.set_ylabel('y')
sub.set_zlabel('z')

sub.view_init(elev=0., azim=0)